In [6]:
#pip install sentence_transformers

In [7]:
from torch.utils.data import DataLoader
from sentence_transformers import models, losses, util
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from datetime import datetime
import logging
import csv
import torch
import tqdm
import sys
import math
import gzip
import os
import pandas as pd

In [8]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [9]:
#You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-basae
model_name = 'roberta-base'
top_k = len(sys.argv) if len(sys.argv) > 2 else 3 #top_k는 전체 문장 후보군의 개수를 넘지 않아야 하므로

batch_size = 16
num_epochs = 1
max_seq_length = 512

In [10]:
###### Read Datasets ######

#Check if dataset exsist. If not, download and extract  it
sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

cross_encoder_path = 'output/cross-encoder/stsb_indomain_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
bi_encoder_path = 'output/bi-encoder/stsb_augsbert_BM25_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [11]:
###### Cross-encoder (simpletransformers) ######
logging.info("Loading sentence-transformers model: {}".format(model_name))
# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for cross-encoder model
cross_encoder = CrossEncoder(model_name, num_labels=1)


###### Bi-encoder (sentence-transformers) ######
logging.info("Loading bi-encoder model: {}".format(model_name))
# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])


2021-10-22 08:50:03 - Loading sentence-transformers model: roberta-base


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

2021-10-22 08:50:17 - Use pytorch device: cpu
2021-10-22 08:50:17 - Loading bi-encoder model: roberta-base


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2021-10-22 08:50:30 - Use pytorch device: cpu


In [12]:
#####################################################################
#
# Step 1: Train cross-encoder model with (gold) STS benchmark dataset
#
#####################################################################

logging.info("Step 1: Train cross-encoder: ({}) with STSbenchmark".format(model_name))

gold_samples = []
dev_samples = []
test_samples = []

2021-10-22 08:50:30 - Step 1: Train cross-encoder: (roberta-base) with STSbenchmark


In [13]:
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1

        if row['split'] == 'dev':
            dev_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
        elif row['split'] == 'test':
            test_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
        else:
            #As we want to get symmetric scores, i.e. CrossEncoder(A,B) = CrossEncoder(B,A), we pass both combinations to the train set
            gold_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
            gold_samples.append(InputExample(texts=[row['sentence2'], row['sentence1']], label=score))

In [14]:
# We wrap gold_samples (which is a List[InputExample]) into a pytorch DataLoader
train_dataloader = DataLoader(gold_samples, shuffle=True, batch_size=batch_size)


# We add an evaluator, which evaluates the performance during training
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name='sts-dev')

# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the cross-encoder model
cross_encoder.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=cross_encoder_path)

2021-10-22 08:50:30 - Warmup-steps: 72


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/719 [00:00<?, ?it/s]

2021-10-22 11:31:57 - CECorrelationEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2021-10-22 11:36:33 - Correlation:	Pearson: 0.9047	Spearman: 0.9015
2021-10-22 11:36:33 - Save model to output/cross-encoder/stsb_indomain_roberta-base-2021-10-22_08-50-03


In [15]:
index_name = "stsb" # index-name should be in lowercase
logging.info("Step 2.1: Generate STSbenchmark (silver dataset) using top-{} bm25 combinations".format(top_k))

unique_sentences = set()

for sample in gold_samples:
    unique_sentences.update(sample.texts)

unique_sentences = list(unique_sentences) # unique sentences
sent2idx = {sentence: idx for idx, sentence in enumerate(unique_sentences)} # storing id and sentence in dictionary
duplicates = set((sent2idx[data.texts[0]], sent2idx[data.texts[1]]) for data in gold_samples) # not to include gold pairs of sentences again


2021-10-22 11:36:35 - Step 2.1: Generate STSbenchmark (silver dataset) using top-3 bm25 combinations


In [16]:
############################################################################
#
# Step 2: Find silver pairs to label
#
############################################################################

#### Top k similar sentences to be retrieved ####
#### Larger the k, bigger the silver dataset ####

logging.info("Step 2.1: Generate STSbenchmark (silver dataset) using pretrained SBERT \
    model and top-{} semantic search combinations".format(top_k))



2021-10-22 11:36:37 - Step 2.1: Generate STSbenchmark (silver dataset) using pretrained SBERT     model and top-3 semantic search combinations


In [17]:
silver_data = []
sentences = set()
# df_claim_oneclaim = pd.read_csv ('/home/ubuntu/deeppatentsimilarity/patentdata/df_claims_oneclaim_500.csv', encoding="ISO-8859-1")
#df_claim_oneclaim = pd.read_csv ('/home/ubuntu/deeppatentsimilarity/patentdata/df_claims_oneclaim_1000.csv', encoding="ISO-8859-1")
#df_claim_oneclaim['text'] = df_claim_oneclaim['text'].str[3:]
#sentences = df_claim_oneclaim.text.to_list()
#sent2idx = dict(zip(df_claim_oneclaim.text, df_claim_oneclaim.patent_id))
# duplicates  = pd.read_csv ('/home/ubuntu/deeppatentsimilarity/patentdata/duplicates_4.csv', encoding="ISO-8859-1")

for sample in gold_samples:
    sentences.update(sample.texts)

sentences = list(sentences) # unique sentences
sent2idx = {sentence: idx for idx, sentence in enumerate(sentences)} # storing id and sentence in dictionary
duplicates = set((sent2idx[data.texts[0]], sent2idx[data.texts[1]]) for data in gold_samples) # not to include gold pairs of sentences again


In [18]:
# For simplicity we use a pretrained model
semantic_model_name = 'roberta-base-nli-stsb-mean-tokens'
semantic_search_model = SentenceTransformer(semantic_model_name)
logging.info("Encoding unique sentences with semantic search model: {}".format(semantic_model_name))


2021-10-22 11:36:37 - Load pretrained SentenceTransformer: roberta-base-nli-stsb-mean-tokens
2021-10-22 11:36:55 - Use pytorch device: cpu
2021-10-22 11:36:55 - Encoding unique sentences with semantic search model: roberta-base-nli-stsb-mean-tokens


In [19]:
# encoding all unique sentences present in the training dataset
embeddings = semantic_search_model.encode(sentences, batch_size=batch_size, convert_to_tensor=True)

Batches:   0%|          | 0/661 [00:00<?, ?it/s]

In [20]:
logging.info("Retrieve top-{} with semantic search model: {}".format(top_k, semantic_model_name))

2021-10-22 11:49:37 - Retrieve top-3 with semantic search model: roberta-base-nli-stsb-mean-tokens


In [21]:
from tqdm.notebook import tqdm

In [22]:
silver_data = []
sentences = set()

for sample in gold_samples:
    sentences.update(sample.texts)

sentences = list(sentences) # unique sentences
sent2idx = {sentence: idx for idx, sentence in enumerate(sentences)} # storing id and sentence in dictionary
duplicates = set((sent2idx[data.texts[0]], sent2idx[data.texts[1]]) for data in gold_samples) # not to include gold pairs of sentences again

# retrieving top-k sentences given a sentence from the dataset
for idx in tqdm(range(len(sentences)),unit="docs"):
    sentence_embedding = embeddings[idx]
    cos_scores = util.pytorch_cos_sim(sentence_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    top_results = torch.topk(cos_scores, k=top_k+1)
    for score, iid in zip(top_results[0], top_results[1]):
        if iid != idx and (iid, idx) not in duplicates:
            silver_data.append((sentences[idx], sentences[iid]))
            duplicates.add((idx,iid))


  0%|          | 0/10566 [00:00<?, ?docs/s]

In [23]:
logging.info("Length of silver_dataset generated: {}".format(len(silver_data)))
logging.info("Step 2.2: Label STSbenchmark (silver dataset) with cross-encoder: {}".format(model_name))
cross_encoder = CrossEncoder(cross_encoder_path)
silver_scores = cross_encoder.predict(silver_data)

# All model predictions should be between [0,1]
assert all(0.0 <= score <= 1.0 for score in silver_scores)

2021-10-22 11:57:52 - Length of silver_dataset generated: 31698
2021-10-22 11:57:52 - Step 2.2: Label STSbenchmark (silver dataset) with cross-encoder: roberta-base
2021-10-22 11:57:54 - Use pytorch device: cpu


Batches:   0%|          | 0/991 [00:00<?, ?it/s]

In [24]:
############################################################################################
#
# Step 3: Train bi-encoder model with both STSbenchmark and labeled AllNlI - Augmented SBERT
#
############################################################################################

In [25]:
logging.info("Step 3: Train bi-encoder: {} with STSbenchmark (gold + silver dataset)".format(model_name))

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark gold and silver train dataset")
silver_samples = list(InputExample(texts=[data[0], data[1]], label=score) for \
    data, score in zip(silver_data, silver_scores))

train_dataset = SentencesDataset(gold_samples + silver_samples, bi_encoder)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
train_loss = losses.CosineSimilarityLoss(model=bi_encoder)

logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

2021-10-22 13:55:39 - Step 3: Train bi-encoder: roberta-base with STSbenchmark (gold + silver dataset)
2021-10-22 13:55:39 - Read STSbenchmark gold and silver train dataset
2021-10-22 13:55:39 - Read STSbenchmark dev dataset


In [26]:
# Configure the training.
warmup_steps = math.ceil(len(train_dataset) * num_epochs / batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the bi-encoder model
bi_encoder.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=bi_encoder_path
          )

2021-10-22 13:55:39 - Warmup-steps: 270


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2700 [00:00<?, ?it/s]

2021-10-22 17:10:25 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 1000 steps:
2021-10-22 17:13:53 - Cosine-Similarity :	Pearson: 0.8759	Spearman: 0.8751
2021-10-22 17:13:53 - Manhattan-Distance:	Pearson: 0.8513	Spearman: 0.8526
2021-10-22 17:13:53 - Euclidean-Distance:	Pearson: 0.8517	Spearman: 0.8533
2021-10-22 17:13:53 - Dot-Product-Similarity:	Pearson: 0.8015	Spearman: 0.8089
2021-10-22 17:13:53 - Save model to output/bi-encoder/stsb_augsbert_BM25_roberta-base-2021-10-22_08-50-03
2021-10-22 20:10:34 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 2000 steps:
2021-10-22 20:14:40 - Cosine-Similarity :	Pearson: 0.8829	Spearman: 0.8809
2021-10-22 20:14:40 - Manhattan-Distance:	Pearson: 0.8614	Spearman: 0.8626
2021-10-22 20:14:40 - Euclidean-Distance:	Pearson: 0.8620	Spearman: 0.8634
2021-10-22 20:14:40 - Dot-Product-Similarity:	Pearson: 0.8175	Spearman: 0.8217
2021-10-22 20:14:40 - Save model to output/b

In [27]:
#################################################################################
#
# Evaluate cross-encoder and Augmented SBERT performance on STS benchmark dataset
#
#################################################################################

# load the stored augmented-sbert model
bi_encoder = SentenceTransformer(bi_encoder_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(bi_encoder, output_path=bi_encoder_path)

2021-10-22 22:44:50 - Load pretrained SentenceTransformer: output/bi-encoder/stsb_augsbert_BM25_roberta-base-2021-10-22_08-50-03
2021-10-22 22:44:52 - Use pytorch device: cpu
2021-10-22 22:44:52 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2021-10-22 22:46:47 - Cosine-Similarity :	Pearson: 0.8554	Spearman: 0.8471
2021-10-22 22:46:47 - Manhattan-Distance:	Pearson: 0.8351	Spearman: 0.8319
2021-10-22 22:46:47 - Euclidean-Distance:	Pearson: 0.8362	Spearman: 0.8326
2021-10-22 22:46:47 - Dot-Product-Similarity:	Pearson: 0.7639	Spearman: 0.7569


0.8471109552894693

In [28]:
####AugSBERT+KNN

In [29]:
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
from tokenizers import Tokenizer
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)
import pandas as pd
import torch
import random
import itertools

In [30]:
F1Measure_list = []
Recall_list = []
Accuracy_list = []
Precision_list = []
Hamming_Loss_list = []

In [31]:
ls c:/

스위치가 틀립니다 - "".


In [32]:
def get_top_n_similar_patents_df(new_claim, claims):

    embedder = SentenceTransformer('output/bi-encoder/stsb_augsbert_BM25_roberta-base-2021-10-22_08-50-03')
    query_embeddings = embedder.encode([new_claim])
    claim_embeddings = embedder.encode(claims)
    top_n = 40
    distances = scipy.spatial.distance.cdist(query_embeddings, claim_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    top_claim_ids = []
    top_claims = []
    top_similarity_scores = []

    # Find the closest 100 patent claims for each new_claim based on cosine similarity
    for idx, distance in results[0:top_n]:
        top_claim_ids.append(patent_id[idx])
        top_claims.append(claims[idx])
        top_similarity_scores.append(round((1-distance), 4))
        print('Patent ID: ' + str(patent_id[idx]))
        print('PubMed Claim: ' + claims[idx])
        print('Similarity Score: ' + "%.4f" % (1-distance))
        print('\n')
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
        'claims': top_claims,
    })
    
    return top_100_similar_patents_df

def F1Measure(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
            continue
        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i])))/ (sum(y_true[i])+sum(y_pred[i]))
    return temp/ y_true.shape[0]

def Recall(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_pred[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
    return temp/ y_true.shape[0]

def Precision(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_true[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
    return temp/ y_true.shape[0]

def Hamming_Loss(y_true, y_pred):
    temp=0
    for i in range(y_true.shape[0]):
        temp += np.size(y_true[i] == y_pred[i]) - np.count_nonzero(y_true[i] == y_pred[i])
    return temp/(y_true.shape[0] * y_true.shape[1])

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]

df_claim_cpc_test = pd.read_csv('C:/Users/강지연/cpc_test_60.csv', encoding='ISO-8859-1')
df_claim_cpc_train = pd.read_csv('C:/Users/강지연/cpc_training_234.csv', encoding='ISO-8859-1')
# df_claim_cpc_train_1000 = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_cpc_all_len_150_200_1000.csv', encoding='ISO-8859-1')
# df_claim_cpc_test = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/prelabel/NewTest/df_1_L_43259_test_100.csv', encoding='ISO-8859-1')
# df_claim_cpc_train = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/prelabel/NewTest/df_1_L_43259.csv', encoding='ISO-8859-1')

claims = list(df_claim_cpc_train.text)
patent_id = list(df_claim_cpc_train.patent_id)

listofpredictdfs = []

In [41]:
def get_top_n_similar_patents_df(new_claim, claims):

    embedder = SentenceTransformer('output/bi-encoder/stsb_augsbert_BM25_roberta-base-2021-10-22_08-50-03')
    query_embeddings = embedder.encode([new_claim])
    claim_embeddings = embedder.encode(claims)
    top_n = 40
    distances = scipy.spatial.distance.cdist(query_embeddings, claim_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    top_claim_ids = []
    top_claims = []
    top_similarity_scores = []

    print("New_claim: " + new_claim + '\n')

    # Find the closest 100 patent claims for each new_claim based on cosine similarity
    for idx, distance in results[0:top_n]:
        top_claim_ids.append(patent_id[idx])
        top_claims.append(claims[idx])
        top_similarity_scores.append(round((1-distance), 4))
        print('Patent ID: ' + str(patent_id[idx]))
        print('PubMed Claim: ' + claims[idx])
        print('Similarity Score: ' + "%.4f" % (1-distance))
        print('\n')
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
        'claims': top_claims,
    })
    
    return top_100_similar_patents_df

In [61]:
len(claims)

240

In [42]:
df = df_claim_cpc_train

In [43]:
df = df.rename(columns={"text_x": "text", "section_id_y": "section_id", "subsection_id_y": "subsection_id","group_id_y": "group_id", "subgroup_id_y": "subgroup_id"})

In [44]:
claims = list(df.text)
patent_id = list(df.patent_id)

In [45]:
new_claim = ['The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.']

In [46]:
new_claim

['The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.']

In [47]:
if os.path.exists('top_100_similar_patents_df.csv'):
    get_top_n_similar_patents_df = pd.read_csv('top_100_similar_patents_df.csv')

else:
    get_top_n_similar_patents_df = get_top_n_similar_patents_df(new_claim[0], claims)

2021-10-23 12:34:06 - Load pretrained SentenceTransformer: output/bi-encoder/stsb_augsbert_BM25_roberta-base-2021-10-22_08-50-03
2021-10-23 12:34:10 - Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

New_claim: The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.

Patent ID: 6888925
PubMed Claim: A method for testing a conference server for use in a telephone network, said conference server having a plurality full duplex channels, each channel including an input to the conference server and an output from the server, said method comprising the steps of: in a first generating step, generating a plu

In [48]:
get_top_n_similar_patents_df.head()

,top_claim_ids,cosine_similarity,claims
0,6888925,0.5478,A method for testing a conference server for u...
1,6888567,0.5451,An electronic photographing apparatus comprisi...
2,6882735,0.5416,A computer processor method of creating an aud...
3,6862742,0.5239,An information distribution apparatus for a CA...
4,6889386,0.5228,A program guide data processing apparatus comp...


In [49]:
result = pd.concat([get_top_n_similar_patents_df, df], axis=1, join='inner')

In [50]:
result.to_csv(r'output/df_claim_cpc_new_claim_result.csv', index = False)

In [51]:
df_claim_cpc_result = pd.read_csv('output/df_claim_cpc_new_claim_result.csv', encoding='ISO-8859-1')

In [52]:
df_claim_cpc_result = result
df_claim_cpc_result.head()

,top_claim_ids,cosine_similarity,claims,ï»¿,patent_id,text,section_id,subsection_id,group_id,subgroup_id,category,sequence,uuid
0,6888925,0.5478,A method for testing a conference server for u...,0,6845797,"A tire comprising: a tread, where said tread i...",['C'],['C08'],['C08L'],['C08L21/00'],['inventional'],[0],3183e92b-ebb6-11ea-a344-121df0c29c1e
1,6888567,0.5451,An electronic photographing apparatus comprisi...,1,6851219,A process for increasing the wetting rate of w...,['C'],['C09'],['C09K'],['C09K17/16'],['inventional'],[0],382c734f-ebbb-11ea-a344-121df0c29c1e
2,6882735,0.5416,A computer processor method of creating an aud...,2,6851417,An ignition device for internal combustion eng...,['H'],['H01'],['H01L'],['H01L2224/48472'],['additional'],[1],267eefaa-ebcb-11ea-a344-121df0c29c1e
3,6862742,0.5239,An information distribution apparatus for a CA...,3,6858711,A compound having the formula: wherein R 1 is ...,['C'],['C07'],['C07H'],['C07H21/00'],['inventional'],[2],380bc86b-ebb7-11ea-a344-121df0c29c1e
4,6889386,0.5228,A program guide data processing apparatus comp...,4,6862379,"An integrated circuit, the integrated circuit ...",['H'],['H01'],['H01S'],['H01S5/0264'],['inventional'],[1],2d002990-ebc5-11ea-a344-121df0c29c1e


In [53]:
df_claim_cpc_result['section_id'] = df_claim_cpc_result['section_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df_claim_cpc_result['subsection_id'] = df_claim_cpc_result['subsection_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df_claim_cpc_result['group_id'] = df_claim_cpc_result['group_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df_claim_cpc_result['subgroup_id'] = df_claim_cpc_result['subgroup_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

<ipython-input-53-dd753f13ff3a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_claim_cpc_result['section_id'] = df_claim_cpc_result['section_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
<ipython-input-53-dd753f13ff3a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_claim_cpc_result['subsection_id'] = df_claim_cpc_result['subsection_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
<ipython-input-53-dd753f13ff3a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_claim_cpc_result['group_id'] = df_claim_cpc_result['group_id'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
<ipython-input-53-dd753f13ff3a>:4: FutureWarning: The default value 

In [54]:
k_similar_patents = df_claim_cpc_result.nlargest(10, ['cosine_similarity'])

In [55]:
k_similar_patents

,top_claim_ids,cosine_similarity,claims,ï»¿,patent_id,text,section_id,subsection_id,group_id,subgroup_id,category,sequence,uuid
0,6888925,0.5478,A method for testing a conference server for u...,0,6845797,"A tire comprising: a tread, where said tread i...",['C'],['C08'],['C08L'],['C08L21/00'],['inventional'],[0],3183e92b-ebb6-11ea-a344-121df0c29c1e
1,6888567,0.5451,An electronic photographing apparatus comprisi...,1,6851219,A process for increasing the wetting rate of w...,['C'],['C09'],['C09K'],['C09K17/16'],['inventional'],[0],382c734f-ebbb-11ea-a344-121df0c29c1e
2,6882735,0.5416,A computer processor method of creating an aud...,2,6851417,An ignition device for internal combustion eng...,['H'],['H01'],['H01L'],['H01L2224/48472'],['additional'],[1],267eefaa-ebcb-11ea-a344-121df0c29c1e
3,6862742,0.5239,An information distribution apparatus for a CA...,3,6858711,A compound having the formula: wherein R 1 is ...,['C'],['C07'],['C07H'],['C07H21/00'],['inventional'],[2],380bc86b-ebb7-11ea-a344-121df0c29c1e
4,6889386,0.5228,A program guide data processing apparatus comp...,4,6862379,"An integrated circuit, the integrated circuit ...",['H'],['H01'],['H01S'],['H01S5/0264'],['inventional'],[1],2d002990-ebc5-11ea-a344-121df0c29c1e
5,6969619,0.4999,A method of endpoint detection during plasma p...,5,6862742,An information distribution apparatus for a CA...,['H'],['H04'],['H04N'],['H04N21/42684'],['inventional'],[0],11dfe8f5-ebbf-11ea-a344-121df0c29c1e
6,6969621,0.4972,"A method for uniformly contaminating samples, ...",6,6868683,A cooling system for cooling racks in a data c...,['H'],['H05'],['H05K'],['H05K7/20836'],['inventional'],[0],2d1b7a64-ebb7-11ea-a344-121df0c29c1e
7,6962884,0.4950,A method for processing integrated circuit dev...,7,6882689,A data coding method comprising: accepting dig...,['H'],['H04'],['H04L'],['H04L27/001'],['inventional'],[0],029dbf84-ebb7-11ea-a344-121df0c29c1e
8,6888482,0.4882,An analog to digital converter (ADC) comprisin...,8,6882704,A radiation source for generating extreme ultr...,['H'],['H05'],['H05G'],['H05G2/003'],['inventional'],[0],053e423e-ebb7-11ea-a344-121df0c29c1e
9,6882723,0.4867,A method of quantifying an automation benefit ...,9,6882707,A method for training an operator for relay re...,['H'],['H04'],['H04M'],['H04M1/64'],['inventional'],[0],0c70cb18-ebb6-11ea-a344-121df0c29c1e


In [56]:
final_result_mod = k_similar_patents.mode()
# final_result_max = k_similar_patents.max()
print('Prediction for section_id of New Patent is: ', final_result_mod.iloc[0]['section_id'])
print('Prediction for section_id of New Patent is: ', k_similar_patents.iloc[0:10]['section_id'])
print('Prediction for subsection_id of New Patent is: ', k_similar_patents.iloc[0:10]['subsection_id'])
print('Prediction for group_id of New Patent is: ', k_similar_patents.iloc[0:10]['group_id'])
print('Prediction for subgroup_id of New Patent is: ', k_similar_patents.iloc[0:10]['subgroup_id'])

Prediction for section_id of New Patent is:  ['H']
Prediction for section_id of New Patent is:  0    ['C']
1    ['C']
2    ['H']
3    ['C']
4    ['H']
5    ['H']
6    ['H']
7    ['H']
8    ['H']
9    ['H']
Name: section_id, dtype: object
Prediction for subsection_id of New Patent is:  0    ['C08']
1    ['C09']
2    ['H01']
3    ['C07']
4    ['H01']
5    ['H04']
6    ['H05']
7    ['H04']
8    ['H05']
9    ['H04']
Name: subsection_id, dtype: object
Prediction for group_id of New Patent is:  0    ['C08L']
1    ['C09K']
2    ['H01L']
3    ['C07H']
4    ['H01S']
5    ['H04N']
6    ['H05K']
7    ['H04L']
8    ['H05G']
9    ['H04M']
Name: group_id, dtype: object
Prediction for subgroup_id of New Patent is:  0         ['C08L21/00']
1         ['C09K17/16']
2    ['H01L2224/48472']
3         ['C07H21/00']
4        ['H01S5/0264']
5      ['H04N21/42684']
6       ['H05K7/20836']
7        ['H04L27/001']
8         ['H05G2/003']
9          ['H04M1/64']
Name: subgroup_id, dtype: object


In [64]:
def get_top_n_similar_patents_df(new_claim, claims):

    embedder = SentenceTransformer('output/bi-encoder/stsb_augsbert_BM25_roberta-base-2021-10-22_08-50-03')
    query_embeddings = embedder.encode([new_claim])
    claim_embeddings = embedder.encode(claims)
    top_n = 40
    distances = scipy.spatial.distance.cdist(query_embeddings, claim_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    top_claim_ids = []
    top_claims = []
    top_similarity_scores = []

    # Find the closest 100 patent claims for each new_claim based on cosine similarity
    for idx, distance in results[0:top_n]:
        top_claim_ids.append(patent_id[idx])
        top_claims.append(claims[idx])
        top_similarity_scores.append(round((1-distance), 4))
        print('Patent ID: ' + str(patent_id[idx]))
        print('PubMed Claim: ' + claims[idx])
        print('Similarity Score: ' + "%.4f" % (1-distance))
        print('\n')
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
        'claims': top_claims,
    })
    
    return top_100_similar_patents_df

def F1Measure(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
            continue
        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i])))/ (sum(y_true[i])+sum(y_pred[i]))
    return temp/ y_true.shape[0]

def Recall(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_pred[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
    return temp/ y_true.shape[0]

def Precision(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_true[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
    return temp/ y_true.shape[0]

def Hamming_Loss(y_true, y_pred):
    temp=0
    for i in range(y_true.shape[0]):
        temp += np.size(y_true[i] == y_pred[i]) - np.count_nonzero(y_true[i] == y_pred[i])
    return temp/(y_true.shape[0] * y_true.shape[1])

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]

df_claim_cpc_test = pd.read_csv('C:/Users/강지연/cpc_test_60.csv', encoding='ISO-8859-1')
df_claim_cpc_train = pd.read_csv('C:/Users/강지연/cpc_training_234.csv', encoding='ISO-8859-1')
# df_claim_cpc_train_1000 = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/df_claim_cpc_all_len_150_200_1000.csv', encoding='ISO-8859-1')
# df_claim_cpc_test = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/prelabel/NewTest/df_1_L_43259_test_100.csv', encoding='ISO-8859-1')
# df_claim_cpc_train = pd.read_csv('/home/ubuntu/deeppatentsimilarity/patentdata/prelabel/NewTest/df_1_L_43259.csv', encoding='ISO-8859-1')

claims = list(df_claim_cpc_train.text)
patent_id = list(df_claim_cpc_train.patent_id)

listofpredictdfs = []

In [65]:
for i in range(len(df_claim_cpc_test)):
    get_top_n_similar_patents_df_predict = get_top_n_similar_patents_df(df_claim_cpc_test.text.iloc[i], claims)
    result = pd.merge(get_top_n_similar_patents_df_predict, df_claim_cpc_train, left_on='top_claim_ids',right_on='patent_id',how='left',suffixes=('_left','_right'))
    locals()["predict_n"+str(i)] = result.copy()
    listofpredictdfs.append("predict_n"+str(i))

df = pd.concat(map(lambda x: eval(x), listofpredictdfs),keys= listofpredictdfs ,axis=0)

2021-10-23 13:39:59 - Load pretrained SentenceTransformer: output/bi-encoder/stsb_augsbert_BM25_roberta-base-2021-10-22_08-50-03
2021-10-23 13:40:02 - Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Patent ID: 6888173
PubMed Claim: A light emitting diode device comprising: a substrate; an LED mounted on the substrate; electrodes provided on the substrate and connected to the LED for applying a current to the LED; an encapsulating resin encapsulating the LED; and a color filter layer provided on the encapsulating resin, the color filter layer comprising three color layers, wherein the three color layers comprise a cyan layer, a magenta layer and a yellow layer.
Similarity Score: 0.7624


Patent ID: 6888520
PubMed Claim: An active matrix type display, comprising: a substrate; and a plurality of pixels arranged in the form of a matrix on the substrate; wherein each of the pixels comprises a light emission device prepared by forming a transparent pixel electrode and a metal pixel electrode on respective surfaces of a light emitting layer, and a driving circuit for controlling a driving current of the light emission device; the driving circuit is formed on the substrate; the light emis

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Patent ID: 6888202
PubMed Claim: An integrated circuit comprising: a latch circuit including, a first inverter including a first PMOS transistor and a first NMOS transistor with a first data node comprising interconnected source/drains (S/D) of the first PMOS and NMOS transistors; a second inverter including a second PMOS transistor and a second NMOS transistor with a second data node comprising interconnected source/drains (S/D) of the second PMOS and NMOS transistors; wherein the gates of the first PMOS and first NMOS transistors are coupled to the second data node; wherein the gates of the second PMOS and second NMOS transistors are coupled to the first data node; an input switch including, an access transistor including a first S/D coupled to the first data node and to the gate of the second PMOS transistor and to the gate of the second NMOS transistor and including a second S/D coupled to a first data access node and including a gate coupled to a first access control node; and a f

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Patent ID: 6886749
PubMed Claim: An image reading apparatus for reading an image on a read medium as attached to an information processing device including at least a display device and an intensity adjustment device for adjusting display intensity of the display device, as freely removable from the device, comprising: a read control device controlling a read of an image; a first notification device notifying the intensity adjustment device of the information processing device of a change in display intensity of the display device depending on a state of the read of an image controlled by said read control device; and a second notification device notifying the information processing device of a change in display direction of a part or all of the image displayed on the display device depending on a state of the read of the image controlled by said read control device.
Similarity Score: 0.6829


Patent ID: 6887096
PubMed Claim: A connector for making electrical connection, comprising: a 

In [172]:
df

top_claim_ids  cosine_similarity  \
predict_n0  0         6888173             0.7624   
            1         6888520             0.7164   
            2         6887126             0.7158   
            3         6964086             0.7140   
            4         6887364             0.7095   
...                       ...                ...   
predict_n59 35        6888173             0.5843   
            36        6888071             0.5838   
            37        6888137             0.5827   
            38        6968958             0.5796   
            39        6888045             0.5788   

                                                           claims  ï»¿  \
predict_n0  0   A light emitting diode device comprising: a su...  115   
            1   An active matrix type display, comprising: a s...  140   
            2   An apparatus for thinning a semiconductor wafe...   37   
            3   A method of manufacturing thin film piezoelect...  210   
            4   A method for plating a metal strip to be used ...   45   
...                                                           ...  ...   
predict_n59 35  A light emitting diode device comprising: a su...  115   
            36  A layout method for supporting a plurality of ...   99   
            37  A scanning charged-particle beam instrument ha...  111   
            38  An apparatus, comprising: a first electronic d...  221   
            39  A method of producing a fibrous material layer...   95   

                patent_id                                               text  \
predict_n0  0     6888173  A light emitting diode device comprising: a su...   
            1     6888520  An active matrix type display, comprising: a s...   
            2     6887126  An apparatus for thinning a semiconductor wafe...   
            3     6964086  A method of manufacturing thin film piezoelect...   
            4     6887364  A method for plating a metal strip to be used ...   
...                   ...                                                ...   
predict_n59 35    6888173  A light emitting diode device comprising: a su...   
            36    6888071  A layout method for supporting a plurality of ...   
            37    6888137  A scanning charged-particle beam instrument ha...   
            38    6968958  An apparatus, comprising: a first electronic d...   
            39    6888045  A method of producing a fibrous material layer...   

               section_id subsection_id  group_id         subgroup_id  \
predict_n0  0       ['H']       ['H01']  ['H01L']      ['H01L33/507']   
            1       ['H']       ['H01']  ['H01L']     ['H01L51/5271']   
            2       ['H']       ['H01']  ['H01L']       ['H01L22/26']   
            3       ['H']       ['H01']  ['H01L']      ['H01L41/316']   
            4       ['C']       ['C25']  ['C25D']       ['C25D5/028']   
...                   ...           ...       ...                 ...   
predict_n59 35      ['H']       ['H01']  ['H01L']      ['H01L33/507']   
            36      ['H']       ['H05']  ['H05K']  ['H05K2201/10522']   
            37      ['H']       ['H01']  ['H01J']       ['H01J37/22']   
            38      ['H']       ['H05']  ['H05K']      ['H05K5/0021']   
            39      ['D']       ['D04']  ['D04H']        ['D04H3/14']   

                       category sequence                                  uuid  
predict_n0  0    ['additional']      [1]  0c97c6c1-ebc1-11ea-a344-121df0c29c1e  
            1    ['additional']      [6]  275aea04-ebb8-11ea-a344-121df0c29c1e  
            2   ['inventional']      [0]  245bb0ee-ebc8-11ea-a344-121df0c29c1e  
            3   ['inventional']      [1]  03c12e3b-ebc6-11ea-a344-121df0c29c1e  
            4   ['inventional']      [0]  01317a10-ebce-11ea-a344-121df0c29c1e  
...                         ...      ...                                   ...  
predict_n59 35   ['additional']      [1]  0c97c6c1-ebc1-11ea-a344-121df0c29c1e  
            36  

In [1002]:
df_claim_cpc_test.patent_id.iloc[1]

6969627

In [1057]:
df_claim_cpc_test['true']=1.000
type(df_claim_cpc_test.true)


pandas.core.series.Series

In [946]:
k_similar_patents = df.xs(listofpredictdfs[0]).nlargest(2, ['cosine_similarity'])
k_similar_patents

,top_claim_ids,cosine_similarity,claims,ï»¿,patent_id,text,section_id,subsection_id,group_id,subgroup_id,category,sequence,uuid
0,6888173,0.7624,A light emitting diode device comprising: a su...,115,6888173,A light emitting diode device comprising: a su...,['H'],['H01'],['H01L'],['H01L33/507'],['additional'],[1],0c97c6c1-ebc1-11ea-a344-121df0c29c1e
1,6888520,0.7164,"An active matrix type display, comprising: a s...",140,6888520,"An active matrix type display, comprising: a s...",['H'],['H01'],['H01L'],['H01L51/5271'],['additional'],[6],275aea04-ebb8-11ea-a344-121df0c29c1e


In [967]:
top_k = 2

for k in range(0, top_k):
    top_n = k
    predict = pd.DataFrame(0, index=np.arange(1),columns= ['cosine_similarity'])
    #print(predict)
    for n in range(len(listofpredictdfs)):
        n=0 
        k_similar_patents = df.xs(listofpredictdfs[n]).nlargest(top_n, ['cosine_similarity'])
        #print('k_similar_patents')
        #result_k_similar_patents = pd.DataFrame(df_claim_cpc_test.group_id, columns=['group_id'])
#        result_k_similar_patents = pd.DataFrame(df.cosine_similarity, columns=['cosine_similarity'])
#        print(result_k_similar_patents)
        #print(k_similar_patents)
      #  print(result_k_similar_patents)
        
    result_k_similar_patents = pd.DataFrame(df.cosine_similarity, columns=['cosine_similarity'])
    print(result_k_similar_patents)
#    for i in range(0, top_n):
#        i= 0
#        result_k_similar_patents  = k_similar_patents.iloc[i, 1:]
#print('--', result_k_similar_patents)
        
#    result_k_similar_patents_df = pd.DataFrame(result_k_similar_patents, columns= ['cosine_similarity'])
 #   result_k_similar_patents_df.insert(0, "input_patent_id", df_claim_cpc_test.patent_id, True)
 #   locals()["predict"+str(n)] = result_k_similar_patents_df.copy()
#    predict = pd.concat([predict, locals()["predict"+str(n)]], ignore_index=True)
    #print(predict)
#result_k_similar_patents_df = result_k_similar_patents_df[0:0]
#print(result_k_similar_patents_df)

                cosine_similarity
predict_n0  0              0.7624
            1              0.7164
            2              0.7158
            3              0.7140
            4              0.7095
...                           ...
predict_n59 35             0.5843
            36             0.5838
            37             0.5827
            38             0.5796
            39             0.5788

[2400 rows x 1 columns]
                cosine_similarity
predict_n0  0              0.7624
            1              0.7164
            2              0.7158
            3              0.7140
            4              0.7095
...                           ...
predict_n59 35             0.5843
            36             0.5838
            37             0.5827
            38             0.5796
            39             0.5788

[2400 rows x 1 columns]


In [968]:
df.head()

top_claim_ids  cosine_similarity  \
predict_n0 0        6888173             0.7624   
           1        6888520             0.7164   
           2        6887126             0.7158   
           3        6964086             0.7140   
           4        6887364             0.7095   

                                                         claims  ï»¿  \
predict_n0 0  A light emitting diode device comprising: a su...  115   
           1  An active matrix type display, comprising: a s...  140   
           2  An apparatus for thinning a semiconductor wafe...   37   
           3  A method of manufacturing thin film piezoelect...  210   
           4  A method for plating a metal strip to be used ...   45   

              patent_id                                               text  \
predict_n0 0    6888173  A light emitting diode device comprising: a su...   
           1    6888520  An active matrix type display, comprising: a s...   
           2    6887126  An apparatus for thinning a semiconductor wafe...   
           3    6964086  A method of manufacturing thin film piezoelect...   
           4    6887364  A method for plating a metal strip to be used ...   

             section_id subsection_id  group_id      subgroup_id  \
predict_n0 0      ['H']       ['H01']  ['H01L']   ['H01L33/507']   
           1      ['H']       ['H01']  ['H01L']  ['H01L51/5271']   
           2      ['H']       ['H01']  ['H01L']    ['H01L22/26']   
           3      ['H']       ['H01']  ['H01L']   ['H01L41/316']   
           4      ['C']       ['C25']  ['C25D']    ['C25D5/028']   

                     category sequence                                  uuid  
predict_n0 0   ['additional']      [1]  0c97c6c1-ebc1-11ea-a344-121df0c29c1e  
           1   ['additional']      [6]  275aea04-ebb8-11ea-a344-121df0c29c1e  
           2  ['inventional']      [0]  245bb0ee-ebc8-11ea-a344-121df0c29c1e  
           3  ['inventional']      [1]  03c12e3b-ebc6-11ea-a344-121df0c29c1e  
           4  ['inventional']      [0]  01317a10-ebce-11ea-a344-121df0c29c1e

In [989]:
n1=df.xs(listofpredictdfs[0]).nlargest(3, ['cosine_similarity'])
n1.cosine_similarity.iloc[1]

0.7164

In [1080]:
top_k = 20
i=0
predict = pd.DataFrame(0, index=np.arange(1),columns= ['cosine_similarity'])
for i in range(0,len(listofpredictdfs)):
    n1=df.xs(listofpredictdfs[i]).nlargest(top_k, ['cosine_similarity'])
    n1_result=pd.DataFrame(n1.cosine_similarity.iloc[0:top_k],index=np.arange(top_k),columns=['cosine_similarity'])

    #print(n1_result)
    result_k_similar_patents_df = pd.DataFrame(n1_result, columns= ['cosine_similarity'])
    #print(result_k_similar_patents_df)
    #result_table=result_k_similar_patents_df.insert(0, "test_patent_id", df_claim_cpc_test.patent_id.iloc[i], True)
    #locals()["predict"+str(i)] = result_k_similar_patents_df.copy()
    #print(result_k_similar_patents_df)
    
    data = torch.tensor((result_k_similar_patents_df.to_numpy()).astype(float), dtype=torch.float32)
#    print(data)
    m = nn.Sigmoid()
    output = m(data)
    output = (output>0.65).float()
    #print(output)
    output_df = pd.DataFrame(output, columns=['cosine_similarity']).astype(float)
    #print(output_df)
    y_pred = output_df.to_numpy()
#    print(y_pred)
    y_true = df_claim_cpc_test.iloc[:top_k, 10:].to_numpy()
    #print(y_true)
    
    
  
    F1Measure_list.append(F1Measure(y_true,y_pred))
    Recall_list.append(Recall(y_true,y_pred))
    Accuracy_list.append(Accuracy(y_true, y_pred))
    Precision_list.append(Precision(y_true,y_pred))
    Hamming_Loss_list.append(Hamming_Loss(y_true, y_pred))
    print(i,"번째 F1Measure: ", F1Measure(y_true,y_pred))
    print("Recall: ", Recall(y_true,y_pred))
    print("Accuracy: ", Accuracy(y_true, y_pred))
    print("Precision: ", Precision(y_true,y_pred))
    print("Hamming_Loss: ", Hamming_Loss(y_true, y_pred))


    #predict = pd.concat([predict, locals()["predict"+str(i)]], ignore_index=True)
    #print(predict)
    #result_k_similar_patents_df = result_k_similar_patents_df[0:0]
    
    #predict_t = predict.iloc[:,:1]
    #print(predict_t)
#dict = {'F1Measure': F1Measure_list, 'Recall': Recall_list, 'Accuracy': Accuracy_list, 'Precision': Precision_list, 'Hamming_Loss': Hamming_Loss_list}
#df_ModelMetrics = pd.DataFrame(dict)
#df_ModelMetrics.to_csv('/output/modelpredict/Model_Metrics.csv'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), index = False)
#predict.to_csv(r'/output/modelpredict/predict_result_filteraugsbert.csv'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), index = False)

0 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
1 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
2 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
3 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
4 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
5 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
6 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
7 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
8 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
9 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
10 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_Loss:  0.0
11 번째 F1Measure:  1.0
Recall:  1.0
Accuracy:  1.0
Precision:  1.0
Hamming_L

In [1089]:
dict = {'F1Measure': F1Measure_list, 'Recall': Recall_list, 'Accuracy': Accuracy_list, 'Precision': Precision_list, 'Hamming_Loss': Hamming_Loss_list}
#print(len(F1Measure_list))
#print(len(Recall_list))
#print(len(Accuracy_list))
#print(len(Precision_list))
#print(len(Hamming_Loss_list))

df_ModelMetrics= pd.DataFrame.from_dict(dict, orient='index')
df_ModelMetrics
#df_ModelMetrics.to_csv(r'output/df_claim_cpc_new_claim_result.csv', index = False)
df_ModelMetrics.to_csv(r'output/Model_result_Metrics.csv'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), index = False)


#predict.to_csv(r'/output/predict_result_filteraugsbert.csv'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), index = False)

1104
945
942
942
942


In [1101]:
n1

,top_claim_ids,cosine_similarity,claims,ï»¿,patent_id,text,section_id,subsection_id,group_id,subgroup_id,category,sequence,uuid
0,6888173,0.7624,A light emitting diode device comprising: a su...,115,6888173,A light emitting diode device comprising: a su...,['H'],['H01'],['H01L'],['H01L33/507'],['additional'],[1],0c97c6c1-ebc1-11ea-a344-121df0c29c1e
1,6888520,0.7164,"An active matrix type display, comprising: a s...",140,6888520,"An active matrix type display, comprising: a s...",['H'],['H01'],['H01L'],['H01L51/5271'],['additional'],[6],275aea04-ebb8-11ea-a344-121df0c29c1e
2,6887126,0.7158,An apparatus for thinning a semiconductor wafe...,37,6887126,An apparatus for thinning a semiconductor wafe...,['H'],['H01'],['H01L'],['H01L22/26'],['inventional'],[0],245bb0ee-ebc8-11ea-a344-121df0c29c1e
3,6964086,0.7140,A method of manufacturing thin film piezoelect...,210,6964086,A method of manufacturing thin film piezoelect...,['H'],['H01'],['H01L'],['H01L41/316'],['inventional'],[1],03c12e3b-ebc6-11ea-a344-121df0c29c1e
4,6887364,0.7095,A method for plating a metal strip to be used ...,45,6887364,A method for plating a metal strip to be used ...,['C'],['C25'],['C25D'],['C25D5/028'],['inventional'],[0],01317a10-ebce-11ea-a344-121df0c29c1e


In [1126]:
top_k = 5
i=0
predict = pd.DataFrame(0, index=np.arange(1),columns= ['cosine_similarity'])
for i in range(0,len(listofpredictdfs)):
    n1=df.xs(listofpredictdfs[i]).nlargest(top_k, ['cosine_similarity'])
    n1_result=pd.DataFrame(n1.cosine_similarity.iloc[0:top_k],index=np.arange(top_k),columns=['cosine_similarity'])
    n1_result.insert(0, 'group_id', n1.group_id.iloc[0:top_k], allow_duplicates=False)
    n1_result.insert(0, 'top_claim_ids', n1.top_claim_ids.iloc[0:top_k], allow_duplicates=False)


    result_k_similar_patents_df = pd.DataFrame(n1_result, columns= ['cosine_similarity', 'group_id','top_claim_ids'])
    result_k_similar_patents_df.insert(0, "test_patent_id", df_claim_cpc_test.patent_id.iloc[i], True)
    result_k_similar_patents_df.insert(0, "test_group_id", df_claim_cpc_test.group_id.iloc[i], True)
    locals()["predict"+str(i)] = result_k_similar_patents_df.copy()
    print(result_k_similar_patents_df)
    
result_k_similar_patents_df.to_csv(r'output/Predict_result_filteraugsbert.csv'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S"), index = False)

  test_group_id  test_patent_id  cosine_similarity  group_id  top_claim_ids
0      ['H01L']         6969626             0.7624  ['H01L']        6888173
1      ['H01L']         6969626             0.7164  ['H01L']        6888520
2      ['H01L']         6969626             0.7158  ['H01L']        6887126
3      ['H01L']         6969626             0.7140  ['H01L']        6964086
4      ['H01L']         6969626             0.7095  ['C25D']        6887364
  test_group_id  test_patent_id  cosine_similarity  group_id  top_claim_ids
0      ['H01L']         6969627             0.7851  ['H01L']        6962860
1      ['H01L']         6969627             0.7713  ['H01L']        6962874
2      ['H01L']         6969627             0.7708  ['H01L']        6962845
3      ['H01L']         6969627             0.7562  ['H01L']        6888195
4      ['H01L']         6969627             0.7542  ['H04N']        6961077
  test_group_id  test_patent_id  cosine_similarity  group_id  top_claim_ids
0      ['H01

In [1132]:
df_claim_cpc_test.iloc[:top_k, 5]

0    ['H01L']
1    ['H01L']
2    ['H01L']
3    ['H01L']
4    ['H01L']
Name: group_id, dtype: object

In [1157]:
top_k = 5
i=0
predict1 = pd.DataFrame(0, index=np.arange(1),columns= ['group_id'])
for i in range(0,len(listofpredictdfs)):
    n2=df.xs(listofpredictdfs[i]).nlargest(top_k, ['cosine_similarity'])
    n2_result=pd.DataFrame(n1.group_id.iloc[0:top_k],index=np.arange(top_k),columns=['group_id'])

    #print(n1_result)
    result_group_id_df = pd.DataFrame(n2_result, columns= ['group_id'])
    #print(result_k_similar_patents_df)
    #result_table=result_k_similar_patents_df.insert(0, "test_patent_id", df_claim_cpc_test.patent_id.iloc[i], True)
    #locals()["predict"+str(i)] = result_k_similar_patents_df.copy()
    print(result_group_id_df)
    
    data = torch.tensor((result_group_id_df.to_numpy()))
    data
#    m = nn.Sigmoid()
    
#    output_1 = (output>0.65).float()
    #print(output)
    output_1_df = pd.DataFrame(data, columns=['group_id']).astype(float)
    #print(output_df)
#    y_pred = data.to_numpy()
#    print(y_pred)
#    y_true = df_claim_cpc_test.iloc[:top_k, 5].to_numpy()
    #print(y_true)
    
    

   group_id
0  ['H04N']
1  ['H05K']
2  ['H04M']
3  ['H04M']
4  ['H03K']


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.